In [1]:
#import library
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from sklearn.metrics import confusion_matrix
import seaborn as sns
from keras.preprocessing import image
from tensorflow.keras.models import load_model
import os
import pandas as pd
from sklearn.model_selection import StratifiedKFold , KFold ,RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#load train file csv
train = pd.read_csv('/content/drive/MyDrive/Project Siscer/train-salin.csv')
train.head()

,Image,Class
0,1.jpg,mentah
1,2.jpg,mentah
2,3.jpg,mentah
3,4.jpg,mentah
4,5.jpg,mentah


In [4]:
#defined df variable
df = train

In [5]:
#define train dataset path
TRAIN_PATH = '/content/drive/MyDrive/Project Siscer/Dataset/train'

In [6]:
def get_model(IMG_SIZE):
    model = tf.keras.models.Sequential([
    #first_convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    #second_convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    #third_convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    #fourth_convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
#     model.summary()

In [7]:
# Storing the average of all predictions
main_pred = []
error = []
data_kfold = pd.DataFrame()

In [8]:
# Creating X, Y for training
train_y = df.Class
train_x = df.drop(['Class'],axis=1)

In [13]:
IMG_SIZE = 150
BATCH_SIZE = 32
EPOCHS = 100
N_SPLIT = 5

In [14]:
# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []

In [ ]:
#Initializing Data Generators
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
validation_datagen = ImageDataGenerator(rescale = 1./255)

# k-fold
kfold = StratifiedKFold(n_splits=N_SPLIT,shuffle=True,random_state=42)

# Variable for keeping count of split we are executing
j = 0

# K-fold Train and test for each split
for train_idx, val_idx in list(kfold.split(train_x,train_y)):
    x_train_df = df.iloc[train_idx]
    x_valid_df = df.iloc[val_idx]
    j+=7


    training_set = train_datagen.flow_from_dataframe(dataframe=x_train_df, directory=TRAIN_PATH,
                                                 x_col="Image", y_col="Class",
                                                 class_mode="categorical",
                                                 target_size=(IMG_SIZE,IMG_SIZE), batch_size=BATCH_SIZE)

    validation_set = validation_datagen.flow_from_dataframe(dataframe=x_valid_df, directory=TRAIN_PATH,
                                                 x_col="Image", y_col="Class",
                                                 class_mode="categorical",
                                                 shuffle=False,
                                                 target_size=(IMG_SIZE,IMG_SIZE), batch_size=BATCH_SIZE)

    model = get_model(IMG_SIZE)


    history = model.fit(training_set,
                        validation_data=validation_set,
                        epochs = EPOCHS,
                        steps_per_epoch=x_train_df.shape[0] // 3 / BATCH_SIZE
                        )

    plt.plot(history.history['loss'], label='Loss')
    plt.plot(history.history['accuracy'], label='Accuracy')
    plt.title(f'Model Accuracy and Loss Fold {j}')
    plt.ylabel('value')
    plt.xlabel('No. epoch')
    plt.legend(loc="upper left")
    plt.show()

    scores = model.evaluate(validation_set, verbose=0)

    pred = model.predict(validation_set)
    y_pred = np.argmax(pred, axis=1)

    cf_matrix = confusion_matrix(validation_set.classes, y_pred)
    import seaborn as sns
    f, ax = plt.subplots(figsize=(9, 6))
    sns.heatmap(cf_matrix, annot=True, fmt="d", linewidths=.5, ax=ax)
    plt.xlabel("y_pred")
    plt.ylabel("y_true")
    ax.xaxis.set_ticklabels(['mentah', 'setengah-matang','matang','busuk'])
    ax.yaxis.set_ticklabels(['mentah', 'setengah-matang','matang','busuk'])
    plt.show()

    model.save(f'/content/drive/MyDrive/Project Siscer/model/model{j}.h5')
    print(f'Score for fold {j}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

In [ ]:
# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} ')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

In [ ]:
#ploting k-fold accuracy and loss
from matplotlib import style

plt.plot(acc_per_fold, label='Accuracy')
plt.plot(loss_per_fold, label='Loss')
plt.title('K-Fold Accuracy and Loss')
plt.ylabel('value')
x = ["Fold-1", "Fold-2", "Fold-3", "Fold-4", "Fold-5"]

default_x_ticks = range(len(x))
plt.xticks(default_x_ticks, x)
plt.legend(loc="upper left")
plt.style.use('ggplot')
plt.show()


In [ ]:
model = load_model('/content/drive/MyDrive/Project Siscer/model-1/model5.h5')

In [ ]:
plt.plot(history.history['loss'], label='Loss')
plt.plot(history.history['accuracy'], label='Accuracy')
plt.title('Model Accuracy and Loss Fold 1')
plt.ylabel('value')
plt.xlabel('No. epoch')
plt.legend(loc="upper left")
plt.show()

scores = model.evaluate(validation_set, verbose=0)

pred = model.predict(validation_set)
y_pred = np.argmax(pred, axis=1)

cf_matrix = confusion_matrix(validation_set.classes, y_pred)
import seaborn as sns
f, ax = plt.subplots(figsize=(9, 6))
sns.heatmap(cf_matrix, annot=True, fmt="d", linewidths=.5, ax=ax)
plt.xlabel("y_pred")
plt.ylabel("y_true")

ax.xaxis.set_ticklabels(['mentah', 'setengah-matang','matang','busuk'])
ax.yaxis.set_ticklabels(['mentah', 'setengah-matang','matang','busuk'])
plt.show()

print(f'Score for fold 1: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
acc_per_fold.append(scores[1] * 100)
loss_per_fold.append(scores[0])